<a href="https://colab.research.google.com/github/Arumat546/nico_cls/blob/main/json_to_csv(SQL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/My Drive/Google Colab/niconico_dataset_add_smid

/content/drive/My Drive/Google Colab/niconico_dataset_add_smid


## **SQLite格納**

In [ ]:
#jsonの読み込みと一覧表示例
import json
import gzip

with gzip.open("3000.json.gz", 'r') as f
     for line in f:
        jsn = json.loads(line)
        print(jsn)


In [ ]:
#json→csv変換
import pandas as pd
import json
import glob

path = "/content/drive/My Drive/Google Colab/niconico_dataset_add_smid/video_dst" # use your path
all_files = glob.glob(path + "/*.json.gz")

li = []

for filename in all_files:
    df = pd.read_json(filename, compression='infer',lines=True)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

frame.to_csv('video.csv', columns=['video_id','category'], index=False)

In [ ]:
#sqliteデータベースに格納(動画csvファイル)
import pandas as pd
import sqlite3
import json
import glob

path = "/content/drive/My Drive/Google Colab/niconico_dataset_add_smid" # use your path
all_files = glob.glob(path + "/video3.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)

conn = sqlite3.connect('2525.db')
c = conn.cursor()
df.to_sql('video', conn, if_exists='append')

conn.commit()
conn.close()

In [ ]:
#sqliteデータベースに格納(コメントjsonファイル)改良版
import pandas as pd
import sqlite3
import json
import glob

conn = sqlite3.connect('2525.db')
c = conn.cursor()

path = "/content/drive/My Drive/Google Colab/niconico_dataset_add_smid/comment_dst" # use your path
all_files = glob.glob(path + "/340[0-3].json.gz")


for filename in all_files:
    df = pd.read_json(filename, compression='infer', orient='records',lines=True)
    df.to_sql('comment', conn,if_exists='append')

conn.commit()
conn.close()

In [ ]:
#一つずつ格納(コメントファイル)
import pandas as pd
import sqlite3
import json

path = "/content/drive/My Drive/Google Colab/niconico_dataset_add_smid/comment_dst" # use your path

df = pd.read_json(path + "/3410.json.gz", compression='infer',lines=True) #昇順でいれること！

conn = sqlite3.connect('2525.db')
c = conn.cursor()
df.to_sql('comment', conn,if_exists='append')
conn.commit()
conn.close()

## **SQL操作**

In [ ]:
#データの削除（カテゴリーがnullのものをすべて削除）
con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("delete from video where category is null")

con.commit()
con.close()

In [ ]:
#列の値の種類抽出
import sqlite3
con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("select category from video group by category")
for row in c: # rowはtuple
    print(row)

con.commit()
con.close()

In [13]:
#ロー数確認
import sqlite3
conn = sqlite3.connect('2525.db')
conect = conn.cursor()

#SQL 設定　　テーブル名：table_ubuntu
sql = 'SELECT count(*) FROM ' + 'comment'
#SQLを実行し、レコード数を得る
conect.execute(sql)
result = conect.fetchall()
#結果は リスト、タプル型で出力される => [(5,)]
print (result)

record_max = result[0][0]
print ('登録されている総レコード数 ==> ', record_max)

[(4544786,)]
登録されている総レコード数 ==>  4544786


In [ ]:
#テーブル一覧取得
import sqlite3
con = sqlite3.connect("2525.db")
cursor = con.cursor()
cursor.execute("select * from sqlite_master where type='table'")
for x in cursor.fetchall():
    print(x)

con.close()

('table', 'video', 'video', 2, 'CREATE TABLE "video" (\n"index" INTEGER,\n  "video_id" TEXT,\n  "category" TEXT\n)')
('table', 'comment', 'comment', 192078, 'CREATE TABLE "comment" (\n"index" INTEGER,\n  "date" TIMESTAMP,\n  "vpos" INTEGER,\n  "content" TEXT,\n  "command" TEXT,\n  "video_id" TEXT\n)')


In [ ]:
#カラム名取得
conn = sqlite3.connect("2525.db")
c = conn.execute('select * from comment')
names = list(map(lambda x: x[0], c.description))
print(names)
conn.close()

['index', 'date', 'vpos', 'content', 'command', 'video_id']


In [ ]:
#テーブル削除
import sqlite3
con = sqlite3.connect('2525.db')
cur = con.cursor()

#sql = "drop table comment_test;"
cur.execute(sql)
 
con.close()

In [ ]:
#内部結合
import sqlite3
i=0
con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("select content,category from comment inner join video on comment.video_id=video.video_id where length(content)>=10 and length(content)<=20 and (category='アイドルマスター'or category='東方')")
for row in c: # rowはtuple
    print(row[0], row[1])
    i+=1

print(i)
con.commit()
con.close()

In [ ]:
%cd /content

/content


In [ ]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合

In [ ]:
#tsv出力
import pandas as pd
import sqlite3

i=0
df = pd.DataFrame(index=[], columns=['content','category'])

con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("select content,category from comment inner join video on comment.video_id=video.video_id where length(content)>=8 and category='アイドルマスター'")
for row in c: # rowはtuple
    #print(row[0], row[1])
    df.loc[i]=[row[0], row[1]]
    i+=1

df.head()
#データをシャッフル
df = df.sample(frac=1, random_state=121).reset_index(drop=True)


#取り出すデータ数指定
df[:2000].to_csv('/content/imas2000.tsv', sep='\t', index=False, header=None)
#print(df[:len].shape)
#df.to_csv('/content/entame.tsv', sep='\t', index=False)
con.commit()
con.close()

## **実験場**

In [12]:
#各コメント数集計
import sqlite3
import csv
i=0
com=[]
num=[]

connection = sqlite3.connect('2525.db')
cursor = connection.cursor()

cursor.execute("select content, count(*) as namecount from comment group by content having(count(*)>1000) order by content DESC")
for row in cursor: # rowはtuple

    print(row[0], row[1])
    com[i]=row[0]
    num[i]=row[1]
    i+=1
#print(i)

connection.commit()
connection.close()

ﾋｴｯ 1195
ｗｗｗｗｗｗｗｗｗｗｗ 1244
ｗｗｗｗｗｗｗｗｗｗ 1494
ｗｗｗｗｗｗｗｗｗ 1892
ｗｗｗｗｗｗｗｗ 2248
ｗｗｗｗｗｗｗ 2628
ｗｗｗｗｗｗ 4164
ｗｗｗｗｗ 6001
ｗｗｗｗ 7702
ｗｗｗ 12516
ｗｗ 4427
ｗ 2699
？？？ 1591
？ 5079
！？ 19685
誰？ 1164
草 82727
知ってた 1443
生きがい 2036
有能 2030
最高 1253
待ってた 1624
好き 1611
可愛い 2740
乙 4085
ファッ！？ 1604
ん？ 10164
わかる 3711
やりますねぇ！ 1289
やったぜ。 3000
やったぜ 3243
やさしい 1978
みえ 1262
ほんへ 1369
ほんとぉ？ 1267
へー 1720
は？ 15177
はい 1732
なるほど 1506
でかい 2054
つよい 2796
それな 1528
そうはならんやろ 1475
そうだよ 1903
せやな 4749
すごい 1433
すこ 2199
すき 4776
こわい 2799
こわ 1187
これすき 1293
ここ好き 1001
ここすこ 1154
ここすき 7383
ぐへへ 2414
かわいそう 1290
かわいい 32762
かっこいい 3433
お？ 1012
お誕生日おめでとうございます！ 1048
おー 1284
おまどうま！ 2037
おまどうま 1002
おまかわ 1074
おつー 4379
おつです 2190
おつでした 1485
おつおつ 2223
おつ 19962
おおおお 1032
おおお 1799
おお 4439
おい 1267
お 3445
え？ 3309
えっ 1349
ええんやで 1210
ええやん 1503
えぇ・・・ 3017
えぇ… 3361
え 3903
うん 1243
うまい 1409
うぽーつ 1042
うぽつ～ 1507
うぽつ！！ 1696
うぽつ！ 7953
うぽつー 8745
うぽつです！！！ 1082
うぽつです！！ 3340
うぽつです！ 15110
うぽつですー 1398
うぽつです 12909
うぽつ 77735
うぽ 2615
いつもの 2759
いいね 2468
あーあ 1082
あー 1520
ありがとう 1252
あら＾～ 

In [ ]:
SELECT Type, COUNT(*) AS NameCount
FROM LIST
GROUP BY Type
HAVING (COUNT(*) > 1)
ORDER BY Type

In [ ]:
import sqlite3 
#特定カテゴリのデータ数確認
i=0
con = sqlite3.connect("nico.db")
c = con.cursor()
c.execute("select * from video where category='音楽'")
for row in c: # rowはtuple
    #print(row)
    i+=1

print(i)
con.commit()
con.close()


1083519


In [ ]:
#特定文字数以上のレコードの取得（select)
con = sqlite3.connect("nico.db")
c = con.cursor()
c.execute("select content,video_id from comment where length(content)>=20 and length(content)<=22")#5文字以上8文字以内
for row in c: # rowはtuple
    print(row)
con.commit()
con.close()